In [1]:
%%capture
%pip install finlab > log.txt
%pip install talib-binary > log.txt
%pip install jupyter-dash

In [25]:

import pandas as pd
import numpy as np
import datetime
import warnings 
warnings.filterwarnings('ignore')

In [26]:
#read excel
df = pd.read_excel('/Users/mac/Desktop/desktop/code/python/stock/transaction5.xlsx', header = 0)

df = df.iloc[:len(df)-1,:11]


In [27]:
# split stock code to new column
columns = df.columns.tolist()

new_col = '代號'
columns.insert(1, new_col)  # 調整欄位順序用
df[new_col] = df.商品.apply(lambda x: x.split("(")[1].split(")")[0])
df.商品 = df.商品.apply(lambda x: x.split("(")[0])
df = df.loc[:,columns]
df['成交日'] = pd.to_datetime(df['成交日'], format="%Y/%m/%d")

In [28]:
for i in range(len(df)):
    money = df.iloc[i,9]
    if money < 0:
        df.iloc[i,9]= money*-1
        df.iloc[i,10]= 0
    else :
        df.iloc[i,9] = 0
        df.iloc[i,10] = money


In [29]:
#計算平均持有日
def days(df):
    df_buy = df[df['交易別'].str.endswith('買進')]
    df_sold = df[df['交易別'].str.endswith('賣出')]
    
    if(len(df_sold) != 0 ): #有賣出才分析
        df_buy = df_buy.sort_values(by=["成交日"])
        df_sold = df_sold.sort_values(by=["成交日"])

        avg_day = []
        # 只可能買進沒賣出 目前沒有融券空
        # 遇到零股 可能會有賣次數>買的可能
        for i in range(min(len(df_buy),len(df_sold))):
            avg_day.append((df_sold.iloc[i,0] - df_buy.iloc[i,0]).days)
        return (np.mean(avg_day))     
    else : 
        return (np.nan)
 

In [30]:
df_buy = df[df['交易別'].str.endswith('買進')]
df_sold = df[df['交易別'].str.endswith('賣出')]
df_buy = df_buy.sort_values(by=["成交日"])
df_sold = df_sold.sort_values(by=["成交日"])
test_buy = df_buy[df_buy['代號'] == '1307']
test_sold = df_sold[df_sold['代號'] == '1307']

#test_buy.iloc[2,:]
templist_s = pd.DataFrame()
templist_s = templist_s.append(test_sold)
#test_sold.append(test_sold)
templist_s.iloc[:,4]

#test_buy.iloc[2,:]
templist_b = pd.DataFrame()
templist_b = templist_b.append(test_buy)
#test_sold.append(test_sold)
len(test_buy[test_buy.iloc[:,4] < 1000]) == 0

True

In [54]:
#計算平均報酬
def portfolio(df):

    df_buy = df[df['交易別'].str.endswith('買進')]
    df_sold = df[df['交易別'].str.endswith('賣出')]

    df_buy.reset_index(drop=True)
    df_sold.reset_index(drop=True)
    # 至少買賣都大於0筆才能計算
    # 零股先不算
    if((len(df_sold) != 0) & (len(df_buy) != 0) & (len(df_buy[df_buy.iloc[:,4] < 1000]) == 0)):

        pos_percent = []
        neg_percent = []

        # 只可能買進沒賣出 目前沒有融券空
        # 遇到零股 可能會有賣次數>買的可能
        # if(sum(df_buy.iloc[:,4]) == sum(df_sold.iloc[:,4])):
        for i in range(len(df_buy)):
            volumn = df_buy.iloc[i,4]/1000
            templist_b = pd.DataFrame()
            # 買大於一張展開
            if(int(volumn)>1):
                count_b=1 #少加一次因為自己就算一次
                temp_b = df_buy.iloc[i,:]              
                while int(count_b) < int(volumn) :
                    #temp_b.iloc[count_b,4]=1000
                    templist_b = templist_b.append(temp_b)
                    count_b += 1
                df_buy = df_buy.append(templist_b)
        for j in range(len(df_sold)):
            volumn = df_sold.iloc[j,4]/1000
            templist_s = pd.DataFrame()
            # 賣大於一張展開
            if(int(volumn)>1):
                count_s=1 #少加一次因為自己就算一次
                temp_s = df_sold.iloc[j,:]
                while int(count_s) < int(volumn) :
                    #temp_s.iloc[count_s,4]=1000
                    templist_s = templist_s.append(temp_s)
                    count_s += 1
                df_sold = df_sold.append(templist_s)

        # 一買一賣 算完刪除 
        df_buy.reset_index(drop=True)
        df_sold.reset_index(drop=True)
        
        df_buy = df_buy.sort_values(by=["成交日"])
        df_sold = df_sold.sort_values(by=["成交日"])

        #
        for i in range(len(df_sold)):            
            buy_price = df_buy.iloc[i,5] #成交價
            sold_price = df_sold.iloc[i,5]
            print("buy_price:"+buy_price)
            print("sold_price:"+sold_price)

            result = (sold_price-buy_price)/buy_price
            print("result:"+result)
            if (result > 0):
                pos_percent.append(result)
            else:
                neg_percent.append(result) 
        #
        return (round(np.mean(pos_percent),5),round(np.mean(neg_percent),5))     
    else : 
        return (np.nan , np.nan)
 

In [63]:
test = df[df['商品']=='廣達']

df_buy = test[test['交易別'].str.endswith('買進')]
df_sold = test[test['交易別'].str.endswith('賣出')]

df_buy.reset_index(drop=True)
df_sold.reset_index(drop=True)
df_buy


,成交日,代號,商品,交易別,數量,成交價,價金,手續費,交易稅,客戶應收付金額,應收金額,融資金額
7,2023-10-03,2382,廣達,現股買進,1000,248.0,248000,353,0,248353,0,0
12,2023-09-28,2382,廣達,現股買進,1000,239.5,239500,341,0,239841,0,0
21,2023-09-21,2382,廣達,現股買進,1000,215.5,215500,307,0,215807,0,0
22,2023-09-21,2382,廣達,現股買進,1000,219.5,219500,312,0,219812,0,0
124,2023-07-24,2382,廣達,現股買進,1000,243.0,243000,346,0,243346,0,0
130,2023-07-20,2382,廣達,現股買進,1000,206.5,206500,294,0,206794,0,0
137,2023-07-18,2382,廣達,現股買進,1000,197.0,197000,280,0,197280,0,0
173,2023-07-04,2382,廣達,現股買進,1000,163.0,163000,232,0,163232,0,0


In [66]:
df_buy.iloc[0,5]

248.0

In [59]:
test.reset_index(drop=True)
test.sort_values(by=["成交日"])
test

,成交日,代號,商品,交易別,數量,成交價,價金,手續費,交易稅,客戶應收付金額,應收金額,融資金額
7,2023-10-03,2382,廣達,現股買進,1000,248.0,248000,353,0,248353,0,0
10,2023-10-03,2382,廣達,現沖賣出,1000,249.5,249500,355,374,0,248771,0
12,2023-09-28,2382,廣達,現股買進,1000,239.5,239500,341,0,239841,0,0
15,2023-09-28,2382,廣達,現沖賣出,1000,236.0,236000,336,354,0,235310,0
21,2023-09-21,2382,廣達,現股買進,1000,215.5,215500,307,0,215807,0,0
22,2023-09-21,2382,廣達,現股買進,1000,219.5,219500,312,0,219812,0,0
27,2023-09-21,2382,廣達,現沖賣出,1000,219.0,219000,312,328,0,218360,0
124,2023-07-24,2382,廣達,現股買進,1000,243.0,243000,346,0,243346,0,0
125,2023-07-24,2382,廣達,現股賣出,1000,241.5,241500,344,724,0,240432,0
128,2023-07-24,2382,廣達,現沖賣出,1000,233.0,233000,332,349,0,232319,0


In [46]:
#先看現股
#df_full = df[(df['交易別'].str.endswith('買進')) & (df['數量'] == 1000)]
avg_price = df.groupby(['代號','商品']).apply(portfolio) 
avg_price = pd.DataFrame(avg_price)
avg_price.columns = ['平均報酬']

pos_list = []
neg_list = []
for i in range(len(avg_price)):
    #取正報酬
    pos = avg_price['平均報酬'][i][0]*100
    pos_list.append(pos)
    #取負報酬
    neg = avg_price['平均報酬'][i][1]*100
    neg_list.append(neg)

avg_price.insert(1, column="正報酬(%)", value = pos_list)
avg_price.insert(2, column="負報酬(%)", value = neg_list)
#avg_price = avg_price.drop(["平均報酬"], axis=1)

#avg_price.index.names
portfolio = {
    "stock_code": avg_price.index.get_level_values(0),
    "stock_name": avg_price.index.get_level_values(1),
    "stock_profit": pos_list,   
    "stock_loss": neg_list,  
}
portfolio_df = pd.DataFrame(portfolio)

In [51]:

portfolio_df[portfolio_df['stock_code']=='2382']



,stock_code,stock_name,stock_profit,stock_loss
22,2382,廣達,9.964,-4.115


In [11]:
avg_days = df.groupby(['代號','商品']).apply(days)  
avg_days = pd.DataFrame(avg_days)
type(avg_days)

avg_days.columns = ['持有天數']

#每檔股票的平均處理天數
avg_days


,,持有天數
代號,商品,
0056,元大高股息,NaN
00728,第一金工業30,NaN
00891,中信關鍵半導體,NaN
00919,群益台灣精選高息,NaN
1219,福壽,0.0
...,...,...
8050,廣積,21.0
8086,宏捷科,7.5
8222,寶一,3.0


In [12]:
#當沖比
aa = np.sum(avg_days==0)/len(avg_days)
'當沖比率' + str(round(aa['持有天數'],3))

'當沖比率0.115'

In [146]:
#先看現股
#df_full = df[(df['交易別'].str.endswith('買進')) & (df['數量'] == 1000)]
avg_price = df.groupby(['代號','商品']).apply(portfolio) 
avg_price = pd.DataFrame(avg_price)
avg_price.columns = ['平均報酬']

pos_list = []
neg_list = []
for i in range(len(avg_price)):
    #取正報酬
    pos = avg_price['平均報酬'][i][0]*100
    pos_list.append(pos)
    #取負報酬
    neg = avg_price['平均報酬'][i][1]*100
    neg_list.append(neg)

avg_price.insert(1, column="正報酬(%)", value = pos_list)
avg_price.insert(2, column="負報酬(%)", value = neg_list)
#avg_price = avg_price.drop(["平均報酬"], axis=1)

In [ ]:

for i in range(len(df_buy)):
    volumn = df_buy.iloc[i,4]/1000
    # 買大於一張展開
    if(int(volumn)>1):
        count_b=0
        templist_b = pd.DataFrame()
        while int(count_b) < int(volumn) :
            temp_b = df_buy.iloc[i,:]
            temp_b.iloc[i,4]=1000
            templist_b = templist_b.append(temp_b)
            count_b += 1

df_buy.append(templist_b)

In [14]:
avg_price.sort_values(by='正報酬(%)', ascending= False).head(10)

,,正報酬(%),負報酬(%)
代號,商品,,
6187,萬潤,32.558,-2.5
3715,定穎投控,24.916,-3.6
5706,鳳凰,18.093,NaN
3583,辛耘,17.782,-1.2
3483,力致,14.589,-11.3
5210,寶碩,13.834,NaN
2383,台光電,11.305,NaN
6282,康舒,9.509,NaN
2382,廣達,9.458,-4.5


In [15]:
avg_price.sort_values(by='負報酬(%)', ascending= True).head(20)

,,正報酬(%),負報酬(%)
代號,商品,,
3035,智原,NaN,-16.2
2353,宏碁,NaN,-12.6
6793,天力離岸,NaN,-12.1
3483,力致,14.589,-11.3
3591,艾笛森,3.148,-11.3
2402,毅嘉,NaN,-10.0
5258,虹堡,NaN,-9.2
1432,大魯閣,NaN,-9.1
2645,長榮航太,NaN,-9.0


In [23]:
df[df['商品']=='世德']

,成交日,代號,商品,交易別,數量,成交價,價金,手續費,交易稅,應付金額,應收金額,融資金額
23,2023-02-14,2066,世德,現賣,1000,85.7,85700,122,257,0,85321,0.0


In [16]:
#2382	廣達	9.458	-4.5
df[df['商品']=='廣達']


,成交日,代號,商品,交易別,數量,成交價,價金,手續費,交易稅,客戶應收付金額,應收金額,融資金額
7,2023-10-03,2382,廣達,現股買進,1000,248.0,248000,353,0,248353,0,0
10,2023-10-03,2382,廣達,現沖賣出,1000,249.5,249500,355,374,0,248771,0
12,2023-09-28,2382,廣達,現股買進,1000,239.5,239500,341,0,239841,0,0
15,2023-09-28,2382,廣達,現沖賣出,1000,236.0,236000,336,354,0,235310,0
21,2023-09-21,2382,廣達,現股買進,1000,215.5,215500,307,0,215807,0,0
22,2023-09-21,2382,廣達,現股買進,1000,219.5,219500,312,0,219812,0,0
27,2023-09-21,2382,廣達,現沖賣出,1000,219.0,219000,312,328,0,218360,0
124,2023-07-24,2382,廣達,現股買進,1000,243.0,243000,346,0,243346,0,0
125,2023-07-24,2382,廣達,現股賣出,1000,241.5,241500,344,724,0,240432,0
128,2023-07-24,2382,廣達,現沖賣出,1000,233.0,233000,332,349,0,232319,0


In [15]:
avg_stock = avg_days.merge(avg_price , how = 'left', on = ['代號','商品'])

In [32]:
a = sum(avg_stock['正報酬(%)'] > 0)
b = sum(avg_stock['負報酬(%)'] < 0)
print(a)
print(b)

print("勝率" , a/(a+b))

# 勝率其實是高的？  所以就是


27
15
勝率 0.6428571428571429


In [16]:
avg_stock.sort_values(by='負報酬(%)', ascending = True).head(60)
avg_stock.sort_values(by='正報酬(%)', ascending = False).head(60)

stat = round(avg_stock.describe())

stat.iloc[[0,1,3,7]]

,持有天數,正報酬(%),負報酬(%)
count,34.0,27.0,15.0
mean,28.0,7.0,-5.0
min,-37.0,0.0,-15.0
max,199.0,27.0,-0.0


In [26]:
df_2020 = df[df['成交日']<='2023-12-31']

day1 = df['成交日']>='2023-01-01' 
day2 = df['成交日']<'2024-01-01' 
df_2021 = df[(day1 & day2)]

In [28]:
avg_days_2021 = df_2021.groupby(['代號','商品']).apply(days)  
avg_days_2021

代號      商品     
006208  富邦台50             NaN
00679B  元大美債20年    199.000000
1101    台泥           3.000000
1268    漢來美食        32.500000
1410    南染                NaN
1605    華新          75.250000
1720    生達          33.000000
1760    寶齡富錦        40.666667
2066    世德                NaN
2308    台達電         35.000000
2332    友訊          37.000000
2382    廣達           3.000000
2455    全新           6.000000
2457    飛宏          29.000000
2614    東森           1.000000
2748    雲品           0.500000
2882    國泰金               NaN
3003    健和興         20.000000
3037    欣興          16.000000
3042    晶技           8.000000
3540    曜越                NaN
3689    湧德           3.500000
3711    日月光投控        6.000000
4190    佐登-KY       10.000000
5258    虹堡          55.750000
5388    中磊          17.000000
6143    振曜          21.333333
6214    精誠           5.000000
6235    華孚          -5.000000
6282    康舒           4.666667
6285    啟碁           3.000000
6288    聯嘉          53.833333
6669    緯穎          19.0

In [22]:
avg_days_2020 = df_2020.groupby(['代號','商品']).apply(days)  
avg_days_2020 = pd.DataFrame(avg_days_2020)

avg_days_2020.columns = ['持有天數']


avg_price_2020 = df_2020.groupby(['代號','商品']).apply(portfolio) 

avg_price_2020 = pd.DataFrame(avg_price_2020)
avg_price_2020.columns = ['平均報酬']


pos_list = []
neg_list = []
for i in range(len(avg_price_2020)):
    #取正報酬
    pos = avg_price_2020['平均報酬'][i][0]*100
    pos_list.append(pos)
    #取負報酬
    neg = avg_price_2020['平均報酬'][i][1]*100
    neg_list.append(neg)

avg_price_2020.insert(1, column="正報酬(%)", value = pos_list)
avg_price_2020.insert(2, column="負報酬(%)", value = neg_list)

avg_price_2020 = avg_price_2020.drop(["平均報酬"], axis=1)

avg_stock_2020 = avg_days_2020.merge(avg_price_2020 , how = 'left', on = ['代號','商品'])

avg_stock_2020.sort_values(by='負報酬(%)', ascending = True).head(60)
avg_stock_2020.sort_values(by='正報酬(%)', ascending = False).head(60)

stat_2020 = round(avg_stock_2020.describe())

stat_2020.iloc[[0,1,3,7]]


NameError: name 'df_2020' is not defined

In [31]:
avg_days_2021 = df_2021.groupby(['代號','商品']).apply(days)  
avg_days_2021 = pd.DataFrame(avg_days_2021)

avg_days_2021.columns = ['持有天數']


avg_price_2021 = df_2021.groupby(['代號','商品']).apply(portfolio) 

avg_price_2021 = pd.DataFrame(avg_price_2021)
avg_price_2021.columns = ['平均報酬']


pos_list = []
neg_list = []
for i in range(len(avg_price_2021)):
    #取正報酬
    pos = avg_price_2021['平均報酬'][i][0]*100
    pos_list.append(pos)
    #取負報酬
    neg = avg_price_2021['平均報酬'][i][1]*100
    neg_list.append(neg)

avg_price_2021.insert(1, column="正報酬(%)", value = pos_list)
avg_price_2021.insert(2, column="負報酬(%)", value = neg_list)

avg_price_2021 = avg_price_2021.drop(["平均報酬"], axis=1)

avg_stock_2021 = avg_days_2021.merge(avg_price_2021 , how = 'left', on = ['代號','商品'])

avg_stock_2021.sort_values(by='負報酬(%)', ascending = True).head(60)
avg_stock_2021.sort_values(by='正報酬(%)', ascending = False).head(60)

stat_2021 = round(avg_stock_2021.describe())

stat_2021.iloc[[0,1,3,7]]

/Users/mac/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/mac/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,持有天數,正報酬(%),負報酬(%)
count,34.0,27.0,15.0
mean,28.0,7.0,-5.0
min,-37.0,0.0,-15.0
max,199.0,27.0,-0.0


In [38]:
ttt = df_2021.groupby(['代號','商品']).apply(days) 
#pd.DataFrame(ttt)

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
pd.DataFrame(ttt)